## The same as in ONNX_tuning.ipynb but for source i took pretraned model from saved checkpoint

In [1]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet.gluon import nn
import numpy as np

ctx = mx.cpu()

In [2]:
def transform(image):
    resized = mx.image.resize_short(image, 224) #minimum 224x224 images
    cropped, crop_info = mx.image.center_crop(resized, (224, 224))
    normalized = mx.image.color_normalize(cropped.astype(np.float32)/255,
                                      mean=mx.nd.array([0.485, 0.456, 0.406]),
                                      std=mx.nd.array([0.229, 0.224, 0.225]))
    # the network expect batches of the form (N,3,224,224)
    transposed = normalized.transpose((2,0,1))  # Transposing from (224, 224, 3) to (3, 224, 224)
    batchified = transposed.expand_dims(axis=0) # change the shape from (3, 224, 224) to (1, 3, 224, 224)
    return batchified

In [3]:
filename = 'test.jpg'
image = mx.image.imread(filename)
normalized = transform(image)

In [4]:
sym, arg_params, aux_params = mx.model.load_checkpoint('mobilenet_v2_1_0', 1)

all_layers = sym.get_internals()
#print(all_layers)

### if you want to get other output layer, uncomment next two rows, look and select

In [5]:
#output_names = all_layers.list_outputs()
#print(output_names)

### and again I create new_args but skip new_aux

In [6]:
layer_name = 'mobilenetv20_features_batchnorm1_fwd'
#layer_name = 'mobilenetv20_output_flatten0_flatten0'

new_sym = all_layers[layer_name+'_output']
new_args = dict({k:arg_params[k] for k in arg_params if k in new_sym.list_arguments()})

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    pre_trained = nn.SymbolBlock(outputs=new_sym, inputs=mx.sym.var('data'))
net_params = pre_trained.collect_params()

for param in new_args:
    if param in net_params:
        net_params[param]._load_init(new_args[param], ctx=ctx)
for param in aux_params:
    if param in net_params:
        net_params[param]._load_init(aux_params[param], ctx=ctx)

net = nn.HybridSequential()
with net.name_scope():
    net.add(pre_trained)
    #net.add(... your own layers ...)

In [7]:
rez=net(normalized)
print(rez.asnumpy().shape)

(1, 1280, 7, 7)


# YES!!!

In [8]:
### Want to look model schema? Uncomment and run next row.
#mx.visualization.plot_network(new_sym,  node_attrs={"shape":"oval","fixedsize":"false"})